<a href="https://colab.research.google.com/github/csch7/CSCI-4170/blob/main/Homework-03/Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from google.colab import drive

def repl_age(age: str) -> int:
  return int(re.findall(r'\d+', age)[0])

drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/diabetic_data.csv")

for c in df.columns:
  pctUnknown = np.sum([df[c] == '?'])/df[c].shape[0]
  print(c, pctUnknown)
  if(pctUnknown > 0.02):
    df.drop(c, axis = 1, inplace = True)
  else:
    df = df[df[c] != '?']

df.drop('diag_1', axis = 1, inplace = True)
df.drop('diag_2', axis = 1, inplace = True)
df.drop('diag_3', axis = 1, inplace = True)
df.replace('Male', 0, inplace = True)
df.replace('Female', 1, inplace = True)
df = df[df['gender'] != 'Unknown/Invalid']
df.replace('No', 0, inplace = True)
df.replace('NO', 2, inplace = True)
df.replace('>30', 1, inplace = True)
df.replace('<30', 0, inplace = True)
df.replace('Down', 1, inplace = True)
df.replace('Steady', 2, inplace = True)
df.replace('Up', 3, inplace = True)
df.replace('Yes', 1, inplace = True)
df.replace('Ch', 1, inplace = True)
df.replace('Norm', 1, inplace = True)
df.replace('>7', 2, inplace = True)
df.replace('>8', 3, inplace = True)
df.replace('>200', 2, inplace = True)
df.replace('>300', 3, inplace = True)
df.fillna(0, inplace = True)
df['age'] = df['age'].apply(repl_age)

tars = df['readmitted']
df.drop('readmitted', axis = 1, inplace = True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
encounter_id 0.0
patient_nbr 0.0
race 0.022335554114340742
gender 0.0
age 0.0
weight 0.9685847925633315
admission_type_id 0.0
discharge_disposition_id 0.0
admission_source_id 0.0
time_in_hospital 0.0
payer_code 0.395574160328597
medical_specialty 0.49082208203132677
num_lab_procedures 0.0
num_procedures 0.0
num_medications 0.0
number_outpatient 0.0
number_emergency 0.0
number_inpatient 0.0
diag_1 0.0002063557573256294
diag_2 0.0035087719298245615
diag_3 0.01128338659407425
number_diagnoses 0.0
max_glu_serum 0.0
A1Cresult 0.0
metformin 0.0
repaglinide 0.0
nateglinide 0.0
chlorpropamide 0.0
glimepiride 0.0
acetohexamide 0.0
glipizide 0.0
glyburide 0.0
tolbutamide 0.0
pioglitazone 0.0
rosiglitazone 0.0
acarbose 0.0
miglitol 0.0
troglitazone 0.0
tolazamide 0.0
examide 0.0
citoglipton 0.0
insulin 0.0
glyburide-metformin 0.0
glipizide-metformin 0.0
glimepiride-piog

<ipython-input-19-0d3aaf61b92d>:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('No', 0, inplace = True)
<ipython-input-19-0d3aaf61b92d>:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('<30', 0, inplace = True)
<ipython-input-19-0d3aaf61b92d>:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcast

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [21]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

dropped_features = []

for i, feature in enumerate(df.columns):
  vif = variance_inflation_factor(df, i)
  print("{f}:\t\t\t{v}".format(f=feature, v=vif))
  if vif > 5 or np.isnan(vif):
    dropped_features.append(feature)

df.drop(dropped_features, axis = 1, inplace = True)

encounter_id:			5.473443759482801
patient_nbr:			4.237815049998638
gender:			2.1436960717829425
age:			14.234082470786854
admission_type_id:			3.373152597283899
discharge_disposition_id:			1.6217233666802717
admission_source_id:			3.7522618038097257
time_in_hospital:			4.541745785574343
num_lab_procedures:			7.035327276058675
num_procedures:			2.0348641268563514
num_medications:			8.352279720259164
number_outpatient:			1.1267069072201716
number_emergency:			1.1529203114975959
number_inpatient:			1.4116897429671191
number_diagnoses:			18.66700721992207
max_glu_serum:			1.4840185936671306
A1Cresult:			1.2899524957886084
metformin:			1.5724984039985466
repaglinide:			1.0422822212164224
nateglinide:			1.0192188032517904
chlorpropamide:			1.004870754360779
glimepiride:			1.1529677916380008
acetohexamide:			1.0002744546096591
glipizide:			1.3950799206016615
glyburide:			1.3788938068617385
tolbutamide:			1.0012426731944901
pioglitazone:			1.1849356151681982
rosiglitazone:			1.1629030402822185

/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


examide:			nan
citoglipton:			nan
insulin:			3.8647294201227878
glyburide-metformin:			1.0231531886379288
glipizide-metformin:			1.0015323666541383
glimepiride-pioglitazone:			1.000142467815177
metformin-rosiglitazone:			1.0003199562807232
metformin-pioglitazone:			1.0003288087671889
change:			3.661770735202716
diabetesMed:			8.245978697329381


In [97]:
class myNeuralNetwork:
  def __init__(self, train_data, train_targets, epochs, num_hidden_layers=2, hidden_dim=16, lr=0.01):
    self.train_data = train_data
    self.train_targets = train_targets
    self.epochs = epochs
    self.learning_rate = lr
    self.hidden_layers = num_hidden_layers
    self.weights = [np.random.uniform(-0.1, 0.1, (np.shape(train_data)[1], hidden_dim))]
    self.biases = [np.random.uniform(-0.1, 0.1, (hidden_dim, 1))]
    for i in range(1, num_hidden_layers):
      self.weights.append(np.random.uniform(-0.1, 0.1, (hidden_dim, hidden_dim)))
      self.biases.append(np.random.uniform(-0.1, 0.1, (hidden_dim, 1)))
    self.weights.append(np.random.uniform(-0.1, 0.1, (hidden_dim, np.shape(train_targets)[1])))
    self.biases.append(np.random.uniform(-0.1, 0.1, (np.shape(train_targets)[1], 1)))

  def forward(self, data):
    z = []
    a = [data.T]
    for l in range(self.hidden_layers+1):
      z.append(np.matmul(self.weights[l].T, a[l]) + self.biases[l])
      if(l != self.hidden_layers):
        a.append(np.tanh(z[l]))
      else:
        a.append(np.exp(z[l])/np.sum(np.exp(z[l]), axis = 0))
    return a

  def backward(self, a):
    deltas = [0] * (self.hidden_layers+1)
    deltas[-1] = 1/(np.shape(self.train_targets)[0])*((a[-1]-self.train_targets.T)*(self.train_targets.T - a[-1])).T
    for l in range(self.hidden_layers-1, -1, -1):
      # print(np.shape(deltas[l+1]), np.shape(self.weights[l+1]), np.shape(a[l+1]))
      deltas[l] = np.matmul(deltas[l+1], self.weights[l+1].T)*(1-np.tanh(a[l+1])**2).T
    return deltas


  def train(self):
    N = np.shape(self.train_data)[0]
    for e in range(self.epochs):
      a = self.forward(self.train_data)
      dels = self.backward(a)
      print(e, self.cost(a[-1], self.train_targets))
      for l in range(self.hidden_layers+1):
        self.weights[l] -= self.learning_rate*(np.matmul(dels[l].T, a[l].T)).T
        self.biases[l] = self.biases[l]-self.learning_rate*np.array([np.mean(dels[l].T, axis=1)]).T

  def cost(self, preds, tars):
    # print(np.shape(preds), np.shape(tars))
    return np.sum(-np.log(preds.T[range(np.shape(preds)[1]), np.argmax(tars, axis=1)]))/np.shape(preds)[1]


In [98]:
dat = np.array(df)
N = np.shape(dat)[0]

pct_train = 0.1
indices = np.arange(N)
np.random.shuffle(indices)

targets = np.zeros((N, 3))
targets[range(N), tars] = 1

train_data = dat[indices[:int(pct_train*N)]]
train_targets = targets[indices[:int(pct_train*N)]]
nn = myNeuralNetwork(train_data, train_targets, 100)
nn.train()

0 1.0876974683599854
1 1.08481087908989
2 1.084205128351775
3 1.0835932590298232
4 1.0829750486675012
5 1.0823502782462808
6 1.0817187323142443
7 1.081080199118305
8 1.0804344707398428
9 1.079781343233577
10 1.0791206167694767
11 1.07845209577749
12 1.0777755890948852
13 1.0770909101159734
14 1.076397876943974
15 1.0756963125447936
16 1.0749860449024604
17 1.0742669071759756
18 1.0735387378573171
19 1.0728013809303527
20 1.0720546860303908
21 1.0712985086041238
22 1.0705327100696944
23 1.069757157976643
24 1.0689717261654728
25 1.068176294926592
26 1.0673707511583794
27 1.0665549885241452
28 1.0657289076077487
29 1.0648924160676554
30 1.0640454287892052
31 1.0631878680348963
32 1.0623196635924812
33 1.0614407529206806
34 1.0605510812923518
35 1.0596506019349208
36 1.058739276167941
37 1.0578170735376164
38 1.056883971948161
39 1.0559399577898576
40 1.054985026063706
41 1.05401918050255
42 1.0530424336885813
43 1.0520548071671323
44 1.0510563315566686
45 1.050047046654915
46 1.049027001